In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as ss

In [3]:
# Price for zero-coupon bond with stochastic interest rate under Vasicek's model
def ZC_Vasicek(F, r, kappa, theta, sigma, t, T):
    
    delta_T = T - t
    
    B = (1 - np.exp(-kappa * delta_T)) / kappa
    
    A = np.exp((theta - (sigma**2) / (2 * kappa**2)) * (B - delta_T) - (sigma**2 / (4 * kappa)) * B**2)
    
    bond_price = F * A * np.exp(-B * r)
    
    return bond_price

In [4]:
print(ZC_Vasicek(F=100, r=0.02, kappa = 0.1, theta = 0.03, sigma=0.02, t=0, T=1))

97.97852638018804


In [5]:
# MC with gamma distribution for the size of losses
def naive_MC_gamma(nr, lam, D, k, th, T):
    h = []
    poissons = np.random.poisson(lam=lam*T, size=nr)
    for i in range(nr):
        x = np.sum(np.random.gamma(shape=k, scale=th, size=poissons[i]))
        h.append(int(x>D))
    return np.cumsum(h)/np.arange(1,nr+1)

In [6]:
def rpoi(a, y, lam, t, T):
    return np.exp(lam * (T-t) * (np.exp(a) - 1) - a * y)

def rgamma(a, y, k, th, n):
    return (1-th*a)**(-k*n) / np.exp(a * y)

In [7]:
# Importance sampling for default probability
def MC_IS_gamma_poi(nr, lam, D, k, th, t, T):
    # Initial variables
    a_poi = np.log(D / (lam * (T-t) * k * th))/2
    poisson_means = lam * (T-t) * np.exp(a_poi)
    a_gamma = 1/th - (k * poisson_means)/D
    new_th = 1/(1/th - a_gamma)

    # Preallocate memory for cumulative sums
    h = np.zeros(nr)
    r_poi = np.zeros(nr)
    r_gamma = np.zeros(nr)
    
    # Loop over the number of simulations
    for i in range(nr):
        # Generate Poisson-distributed count
        loss_n = np.random.poisson(lam=poisson_means)

        # Generate the exponential random variables for this count
        # Directly sum them without creating large intermediate arrays
        x = np.sum(np.random.gamma(shape=k, scale=new_th, size=loss_n))

        # Compute h and r for this simulation
        h[i] = int(x > D)
        r_poi[i] = rpoi(a_poi, loss_n, lam, t, T)
        r_gamma[i] = rgamma(a_gamma, x, k, th, loss_n)

    # Calculate cumulative sum and return the average at each step
    cumulative_sum = np.cumsum(h * r_poi * r_gamma)
    cumulative_avg = cumulative_sum / np.arange(1, nr + 1)

    return cumulative_avg

In [14]:
# Zero-coupon CAT bond pricing
def CAT_ZC_Vasicek(F, r, kappa, theta, sigma, D, lam, k, th, nr, t, T):
    if lam * T * k * th < D:
        price = ZC_Vasicek(F, r, kappa, theta, sigma, t, T)*(1-MC_IS_gamma_poi(nr, lam, D, k, th, t, T)[-1])
    else:  # Case: N == 0 and lam * T * k * th > D
        price = ZC_Vasicek(F, r, kappa, theta, sigma, t, T)*(1-naive_MC_gamma(nr, lam, D, k, th, T)[-1])
    return price

In [16]:
print(CAT_ZC_Vasicek(F=100, r=0.02, kappa=0.1, theta=0.03, sigma=0.02, lam=35, D=9000000000, k=1, th=163500000, nr=5000, t=0, T=1))

96.51899538617474


In [18]:
# CAT bond with coupons pricing
def CAT_C_Vasicek(F, c, r, kappa, theta, sigma, D, lam, k, th, nr, t, N, T):
    c_sum = 0
    dt = T/N

    for coupon_count in range (N):
        if lam * (coupon_count+1) * dt * k * th < D:
            c_sum += ZC_Vasicek(F*c, r, kappa, theta, sigma, t, T=(coupon_count+1)*dt)*(1-MC_IS_gamma_poi(nr, lam, D, k, th, t, T=(coupon_count+1)*dt)[-1])
        else:  # Case: N == 0 and lam * T * k * th > D
            c_sum += ZC_Vasicek(F*c, r, kappa, theta, sigma, t, T=(coupon_count+1)*dt)*(1-naive_MC_gamma(nr, lam, D, k, th, T=(coupon_count+1)*dt)[-1])
    
    if lam * T * k * th < D:
        c_sum = c_sum + ZC_Vasicek(F, r, kappa, theta, sigma, t, T)*(1-MC_IS_gamma_poi(nr, lam, D, k, th, t, T)[-1])
    else:  # Case: N == 0 and lam * T * k * th > D
        c_sum = c_sum + ZC_Vasicek(F, r, kappa, theta, sigma, t, T)*(1-naive_MC_gamma(nr, lam, D, k, th, T)[-1])
    
    return c_sum

In [20]:
print(CAT_C_Vasicek(F=100, c=0.05, r=0.02, kappa=0.1, theta=0.03, sigma=0.02, lam=35, D=9000000000, k=1, th=163500000, nr=5000, t=0, N=3, T=1/4))

114.44812960234674


In [22]:
# Parameter ranges
F = 1000  # Fixed face value
t = 0     # Fixed initial time
#lam = 1   # Fixed Poisson rate
k = 1     # Fixed Gamma shape
th = 163500000    # Fixed Gamma scale
#D = 90000000000    # Fixed threshold
c=0.05
#r=0.03
kappa=0.2
theta=0.03
sigma=0.02
nr = 5000


# lam_values = np.linspace(30, 40, 100)        # Range for lambda
# D_values = np.linspace(7, 11, 21)*1000000000           # Range for threshold
# N_values = [0, 2, 3, 4, 6, 12]      # Range for coupon frequency
# T_values = np.linspace(90, 720, 64)*1/360


In [24]:
from tqdm import tqdm
import csv

N_sim = 600000
N_values = [0, 2, 3, 4, 6, 8, 10, 12]
chunk_size = 10000  
start_chunk = 1  # Allows skipping initial chunks

# Compute total chunks
total_chunks = (N_sim) // chunk_size
results = []

# CSV file path
csv_file = 'CAT_price_gamma.csv'

# Prepare CSV file header if needed
csv_header = ["c", "r", "kappa", "theta", "sigma", "lambda", "D", "N", "T", "Price"]

# Open CSV file in write mode to create the header
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(csv_header)  # Write the header to the CSV file
    
    print(f"Running {N_sim} simulations in {total_chunks} chunks...\n")
    
    for chunk_index in range(1, total_chunks + 1):
        if chunk_index < start_chunk:
            continue  # Skip initial chunks if needed

        chunk_results = []

        for _ in tqdm(range(chunk_size), desc=f"Processing chunk {chunk_index}/{total_chunks}"):
            N = np.random.choice(N_values)  # Randomly select N
            r = np.random.uniform(0, 0.08)
            lam = np.random.uniform(30, 40)
            D = np.random.uniform(7, 13)*1000000000 
            T = np.random.uniform(90, 720)*1/360

            # Condition to determine which function to use
            if N == 0:
                price = CAT_ZC_Vasicek(F, r, kappa, theta, sigma, D, lam, k, th, nr, t, T)
            else:
                price = CAT_C_Vasicek(F, c, r, kappa, theta, sigma, D, lam, k, th, nr, t, N, T)

            chunk_results.append({
                "c": c,
                "r": r,
                "kappa": kappa,
                "theta": theta,
                "sigma": sigma,
                "lambda": lam,
                "D": D,
                "N": N,
                "T": T,
                "Price": price
            })

        # Save the chunk results into the CSV file after each chunk finishes
        for result in chunk_results:
            writer.writerow(result.values())

        results.extend(chunk_results)  # Save chunk results


print("Simulation complete. Results saved to CSV file.")    

Running 600000 simulations in 60 chunks...



Processing chunk 60/60: 100%|█████████████| 10000/10000 [22:03<00:00,  7.55it/s]

Simulation complete. Results saved to CSV file.
